In [1]:
%load_ext autoreload
%autoreload 2

In [34]:
import pandas as pd
import numpy as np
import requests
import datetime
import os
import sys
from copy import copy
sys.path.append('../..')
from data.dataloader import get_covid19india_api_data
from data.processing import get_data, get_district_time_series, combine_districts, concat_sources, get_concat_data

In [3]:
dataframes = get_covid19india_api_data()

In [4]:
original_copy = copy(dataframes)

In [5]:
bglr1 = get_data(dataframes, 'Karnataka', 'Bengaluru Rural')

/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [6]:
bglr2 = get_data(dataframes, 'Karnataka', 'Bengaluru Urban')

In [7]:
bglr_all = combine_districts([bglr1, bglr2], 'Bengaluru')
bglr_all[:3]

,date,hospitalised,total_infected,deceased,recovered,state,district
0,2020-04-24,73,132,4,55,Karnataka,Bengaluru
1,2020-04-25,86,145,4,55,Karnataka,Bengaluru
2,2020-04-26,75,145,5,65,Karnataka,Bengaluru


In [8]:
amd1 = get_district_time_series(dataframes, 'Gujarat', 'Ahmedabad')
amd1[:3]

,date,total_infected,hospitalised,deceased,recovered,district,state
0,2020-03-20,3,0,0,0,Ahmedabad,Gujarat
1,2020-03-21,5,0,0,0,Ahmedabad,Gujarat
2,2020-03-22,8,0,0,0,Ahmedabad,Gujarat


In [9]:
amd2 = get_district_time_series(dataframes, 'Gujarat', 'Ahmedabad', use_dataframe='deaths_recovs')
amd2[:3]

24 deaths and 52 recoveries in Gujarat with unknown district


,date,deceased,recovered,district,state
0,2020-03-29,3,1,Ahmedabad,Gujarat
1,2020-03-30,3,1,Ahmedabad,Gujarat
2,2020-03-31,3,1,Ahmedabad,Gujarat


In [10]:
amd3 = get_data(dataframes, 'Gujarat', 'Ahmedabad')
amd3[:3]

/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,state,district,date,hospitalised,total_infected,deceased,recovered
0,Gujarat,Ahmedabad,2020-04-24,1625,1821,83,113
1,Gujarat,Ahmedabad,2020-04-25,1802,2003,86,115
2,Gujarat,Ahmedabad,2020-04-26,1980,2181,86,115


In [11]:
amd = concat_sources(amd1, amd2, amd3)

In [12]:
bglr3 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Rural')
bglr4 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Urban')
bglr_raw = combine_districts([bglr3, bglr4], 'Bengaluru', 'Karnataka')

In [13]:
bglr5 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Rural', use_dataframe='deaths_recovs')
bglr6 = get_district_time_series(dataframes, 'Karnataka', 'Bengaluru Urban', use_dataframe='deaths_recovs')
bglr_dr = combine_districts([bglr5, bglr6], 'Bengaluru', 'Karnataka')

4 deaths and 149 recoveries in Karnataka with unknown district
4 deaths and 149 recoveries in Karnataka with unknown district


In [14]:
bglr = concat_sources(bglr_raw, bglr_dr, bglr_all)

# fixing dtype issue

In [21]:
amd3.dtypes

state                     object
district                  object
date              datetime64[ns]
hospitalised              object
total_infected            object
deceased                  object
recovered                 object
dtype: object

In [24]:
df_districts = copy(dataframes['df_districts'])
df_districts.dtypes

state        object
district     object
date         object
active       object
confirmed    object
deceased     object
recovered    object
notes        object
dtype: object

In [26]:
state = 'Gujarat'
district = 'Ahmedabad'

In [33]:
df_district = df_districts[np.logical_and(df_districts['state'] == state, df_districts['district'] == district)]
del df_district['notes']
df_district.loc[:, 'date'] = pd.to_datetime(df_district.loc[:, 'date'])
df_district = df_district.loc[df_district['date'] >= '2020-04-24', :]
df_district = df_district.loc[df_district['date'] < datetime.date.today().strftime("%Y-%m-%d"), :]
df_district.columns = [x if x != 'active' else 'hospitalised' for x in df_district.columns]
df_district.columns = [x if x != 'confirmed' else 'total_infected' for x in df_district.columns]
df_district.reset_index(inplace=True, drop=True)
cols = list(set(df_district.columns) - {'state', 'district', 'date'})
df_district[cols] = df_district[cols].apply(pd.to_numeric)
df_district.dtypes

state                     object
district                  object
date              datetime64[ns]
hospitalised               int64
total_infected             int64
deceased                   int64
recovered                  int64
dtype: object

In [35]:
c = get_concat_data(dataframes, district=['East Delhi', 'South West Delhi', 'West Delhi', 'North Delhi', 'New Delhi', 'South Delhi', 'North East Delhi', 'North West Delhi', ''], state='Delhi', new_district_name='Delhi')

54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district
54 deaths and 877 recoveries in Delhi with unknown district


In [36]:
c

,date,total_infected,hospitalised,deceased,recovered,state,district
0,2020-03-02,1,0,0,0,Delhi,Delhi
1,2020-03-05,1,0,0,0,Delhi,Delhi
2,2020-03-06,2,0,0,0,Delhi,Delhi
3,2020-03-07,2,0,0,0,Delhi,Delhi
4,2020-03-08,2,0,0,0,Delhi,Delhi
...,...,...,...,...,...,...,...
60,2020-05-03,426,426,0,0,Delhi,Delhi
61,2020-05-04,426,426,0,0,Delhi,Delhi
62,2020-05-05,426,426,0,0,Delhi,Delhi
63,2020-05-06,426,425,1,0,Delhi,Delhi


In [37]:
delhi_dist = get_data(dataframes, 'Delhi', 'New Delhi')

/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [38]:
delhi_dist

,state,district,date,hospitalised,total_infected,deceased,recovered
0,Delhi,New Delhi,2020-04-24,37,37,0,0
1,Delhi,New Delhi,2020-04-25,37,37,0,0
2,Delhi,New Delhi,2020-04-26,37,37,0,0
3,Delhi,New Delhi,2020-04-27,37,37,0,0
4,Delhi,New Delhi,2020-04-28,37,37,0,0
5,Delhi,New Delhi,2020-04-29,37,37,0,0
6,Delhi,New Delhi,2020-04-30,37,37,0,0
7,Delhi,New Delhi,2020-05-01,37,37,0,0
8,Delhi,New Delhi,2020-05-02,37,37,0,0
9,Delhi,New Delhi,2020-05-03,37,37,0,0


In [39]:
delhi_dist2 = get_data(dataframes, 'Delhi', '')

In [40]:
delhi_dist2

,state,district,date,hospitalised,total_infected,deceased,recovered


In [41]:
for d in ['East Delhi', 'South West Delhi', 'West Delhi', 'North Delhi', 'New Delhi', 'South Delhi', 'North East Delhi', 'North West Delhi', '']:
    print(get_data(dataframes, 'Delhi', d))

    state    district       date hospitalised total_infected deceased  \
0   Delhi  East Delhi 2020-04-24           38             38        0   
1   Delhi  East Delhi 2020-04-25           38             38        0   
2   Delhi  East Delhi 2020-04-26           38             38        0   
3   Delhi  East Delhi 2020-04-27           38             38        0   
4   Delhi  East Delhi 2020-04-28           38             38        0   
5   Delhi  East Delhi 2020-04-29           38             38        0   
6   Delhi  East Delhi 2020-04-30           38             38        0   
7   Delhi  East Delhi 2020-05-01           38             38        0   
8   Delhi  East Delhi 2020-05-02           38             38        0   
9   Delhi  East Delhi 2020-05-03           38             38        0   
10  Delhi  East Delhi 2020-05-04           38             38        0   
11  Delhi  East Delhi 2020-05-05           38             38        0   
12  Delhi  East Delhi 2020-05-06           38      

/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/Users/vishwa/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:1048: SettingWithCopyWarning: 
A value is trying to be set on 

In [42]:
dataframes.keys()

dict_keys(['df_tested', 'df_statewise', 'df_india_time_series', 'df_districtwise', 'df_raw_data_old', 'df_raw_data', 'df_deaths_recoveries', 'df_districts', 'df_travel_history', 'df_resources'])

In [52]:
statewise = dataframes['df_districts']
statewise = statewise[statewise['state'] == 'Delhi']
statewise['district'].unique()

array(['East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi',
       'North West Delhi', 'South Delhi', 'South West Delhi',
       'West Delhi', 'Unknown', 'Central Delhi', 'Shahdara',
       'South East Delhi'], dtype=object)

In [ ]:
'East Delhi', 'New Delhi', 'North Delhi', 'North East Delhi','North West Delhi', 'South Delhi', 'South West Delhi','West Delhi', 'Unknown', 'Central Delhi', 'Shahdara','South East Delhi']